### 准备环境
确保安装依赖：pip install torch jieba matplotlib。
准备一个 train.txt 文件，放入足够多的中文文本（建议至少几百个 token）。

In [1]:
pip install ipykernel


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch jieba matplotlib


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 第一步：导入工具库
这一步我们加载训练 GPT-2 所需的 Python 库：
- **PyTorch**：深度学习框架，提供张量计算和神经网络模块。
- **jieba**：中文分词工具，将句子拆成词语。
- **matplotlib**：绘图库，用于可视化训练过程中的损失变化。

**关键点**：这些库是基础设施，确保我们能处理数据、构建模型并监控训练。

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import jieba
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# 设置随机种子，确保实验可复现
torch.manual_seed(42)
print("工具库导入完成！")

工具库导入完成！


### 第二步：定义超参数
超参数是模型训练的“设置旋钮”，控制模型的大小和训练行为：
- **VOCAB_SIZE**：词汇表大小，决定模型能识别多少个词。
- **EMBED_SIZE**：嵌入维度，每个词用多少数字表示其特征。
- **NUM_HEADS**：多头注意力的头数，控制注意力机制的并行能力。
- **BLOCK_SIZE**：序列长度，一次处理多少个词。
- **STRIDE**：滑动窗口步幅，决定数据切分的间隔。
- **BATCH_SIZE**：批次大小，一次训练多少个序列。
- **NUM_LAYERS**：Transformer 层数，决定模型深度。
- **DROPOUT**：丢弃率，防止模型过拟合。
- **LEARNING_RATE**：学习率，控制优化步长。
- **EPOCHS**：训练轮数，决定训练多久。

**原理**：这些参数直接影响模型的容量和学习效率。Transformer 的灵活性在于通过调整这些值，可以适应不同规模的任务。

In [5]:
VOCAB_SIZE = 1000       # 词汇表大小
EMBED_SIZE = 64         # 嵌入维度
NUM_HEADS = 4           # 多头注意力头数
BLOCK_SIZE = 32         # 序列长度
STRIDE = 4              # 滑动窗口步幅
BATCH_SIZE = 8          # 批次大小
NUM_LAYERS = 2          # Transformer 层数
DROPOUT = 0.1           # Dropout 比例
LEARNING_RATE = 0.001   # 学习率
EPOCHS = 50             # 训练轮数

print(f"超参数设置完成：词汇表={VOCAB_SIZE}，序列长度={BLOCK_SIZE}，训练轮数={EPOCHS}")

超参数设置完成：词汇表=1000，序列长度=32，训练轮数=50


### 第三步：构建词汇表
词汇表是将文本转换为数字的“字典”：
- **分词**：用 jieba 将中文文本拆成词语。
- **统计词频**：计算每个词出现的次数，选出最常见的 1000 个。
- **编号**：给每个词分配一个唯一索引，未知词用 `<UNK>` 表示。

**原理**：语言模型处理的是数字，而非文字。词汇表是桥梁，将人类语言映射到模型能理解的数字空间。

In [6]:
def build_vocab_from_file(file_path, max_vocab_size=VOCAB_SIZE):
    """
    从文本文件构建词汇表
    :param file_path: 训练文件路径
    :param max_vocab_size: 最大词汇表大小
    :return: 词汇表字典 {词: 索引}
    """
    word_freq = Counter()
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            tokens = jieba.cut(line.strip(), cut_all=False)  # 分词
            word_freq.update(tokens)
    
    # 选取高频词，留位置给 <UNK>
    vocab = {word: idx + 1 for idx, (word, _) in enumerate(word_freq.most_common(max_vocab_size - 1))}
    vocab['<UNK>'] = 0
    print(f"词汇表大小: {len(vocab)}")
    return vocab

def tokens_to_indices(tokens, vocab):
    """
    将词序列转换为索引序列
    :param tokens: 分词后的词列表
    :param vocab: 词汇表
    :return: 索引列表
    """
    return [vocab.get(token, vocab['<UNK>']) for token in tokens]

### 第四步：准备训练数据（数据集）
数据集是将文本切成小块的过程：
- **滑动窗口**：从文本中以固定长度（BLOCK_SIZE）切出序列，步幅（STRIDE）决定每次移动多少。
- **自回归任务**：每个序列分为输入和目标，目标是输入的“下一个词”。例如，输入 `[1, 2, 3]`，目标 `[2, 3, 4]`。
- **重合率**：步幅越小，序列间重合越多，数据利用率越高。

**原理**：GPT-2 通过自回归任务（预测下一个词）学习语言模式。滑动窗口确保模型能看到足够的上下文，同时控制数据量。

In [7]:
class TextDataset(Dataset):
    def __init__(self, file_path, block_size, stride, vocab):
        """
        初始化数据集
        :param file_path: 训练文件路径
        :param block_size: 序列长度
        :param stride: 滑动窗口步幅
        :param vocab: 词汇表
        """
        self.block_size = block_size
        self.stride = stride
        self.vocab = vocab
        self.data = []
        
        # 读取并分词
        all_tokens = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                tokens = list(jieba.cut(line.strip(), cut_all=False))
                all_tokens.extend(tokens)
        
        # 转换为索引序列
        indices = tokens_to_indices(all_tokens, vocab)
        
        # 滑动窗口切分
        for i in range(0, len(indices) - block_size, stride):
            seq = indices[i:i + block_size + 1]
            self.data.append(seq)
        
        print(f"总序列数: {len(self.data)}，重合率: {(block_size - stride) / block_size:.2%}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.data[idx]
        return (torch.tensor(seq[:-1], dtype=torch.long),  # 输入序列
                torch.tensor(seq[1:], dtype=torch.long))   # 目标序列

### 第五步：添加位置编码
Transformer 不像传统模型（如 RNN）能天然记住词的顺序，它需要显式地加入位置信息：
- **位置编码**：用数学公式（正弦和余弦函数）为每个词生成一个独特的“位置标记”。
- **为什么需要**：没有位置编码，Transformer 会把句子当成一堆无序的词，无法理解“人工智能”和“技术”的前后关系。

**设计精髓**：位置编码是固定的，巧妙利用三角函数的周期性，确保不同位置的标记既有区别又有规律，完美适配注意力机制。

In [8]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, block_size):
        super().__init__()
        pe = torch.zeros(block_size, embed_size)
        for pos in range(block_size):
            for i in range(0, embed_size, 2):
                pe[pos, i] = math.sin(pos / (10000 ** (i / embed_size)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** (i / embed_size)))
        self.register_buffer('pe', pe)  # 固定参数，不参与训练

    def forward(self, x):
        return x + self.pe[:x.size(1), :]  # 将位置编码加到输入上

### 第六步：创建嵌入层
嵌入层将词索引转换为有意义的数字表示：
- **词嵌入**：每个词被映射到一个高维向量（EMBED_SIZE），表示其语义特征。
- **位置编码**：在词嵌入上叠加位置信息，确保模型知道词的顺序。

**原理**：嵌入层是 Transformer 的输入端，将离散的词转化为连续的向量空间，为后续的注意力计算铺路。

In [9]:
class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size, embed_size, block_size):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_size)  # 词嵌入层
        self.pos_enc = PositionalEncoding(embed_size, block_size)  # 位置编码

    def forward(self, x):
        x = self.token_emb(x)  # 词转为向量
        return self.pos_enc(x)  # 加上位置信息

### 第七步：构建 Transformer Block
Transformer Block 是模型的核心计算单元：
- **多头自注意力**：像多个侦探同时分析句子，从不同角度找出词之间的关系（例如“人工智能”和“技术”相关）。
- **因果掩码**：屏蔽未来的词，确保模型只看当前和之前的词，符合自回归任务（预测下一个词）。
- **前馈网络**：对每个词的表示做进一步加工，增强模型的表达能力。
- **层归一化**：稳定训练过程，避免数值过大或过小。

**设计精髓**：多头注意力是 Transformer 的“超级大脑”，能并行捕捉多种依赖关系；因果掩码保证了时间顺序的正确性。

In [10]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, num_heads, block_size, dropout):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_size, num_heads, dropout=dropout)  # 多头自注意力
        self.norm1 = nn.LayerNorm(embed_size)  # 层归一化
        self.mlp = nn.Sequential(
            nn.Linear(embed_size, 4 * embed_size),  # 扩展维度
            nn.ReLU(),                              # 激活函数
            nn.Linear(4 * embed_size, embed_size),  # 压缩回去
            nn.Dropout(dropout),                    # 随机丢弃，防过拟合
        )
        self.norm2 = nn.LayerNorm(embed_size)
        self.register_buffer("mask", torch.triu(torch.ones(block_size, block_size), diagonal=1).bool())  # 因果掩码

    def forward(self, x):
        seq_len = x.size(0)
        attn_output, _ = self.attn(x, x, x, attn_mask=self.mask[:seq_len, :seq_len])  # 自注意力计算
        x = self.norm1(x + attn_output)  # 残差连接 + 归一化
        mlp_output = self.mlp(x)         # 前馈网络
        x = self.norm2(x + mlp_output)   # 再次残差连接 + 归一化
        return x

### 第八步：组装 MiniGPT
MiniGPT 是简化的 GPT-2 模型：
- **嵌入层**：将词索引转为向量表示。
- **Transformer Blocks**：多个 Block 叠加，逐层提炼语言特征。
- **输出头**：将最终表示映射回词汇表，预测下一个词的概率。

**原理**：多层 Transformer Block 的叠加增强了模型的深度，能捕捉更复杂的语言模式；残差连接和归一化确保训练稳定。

In [11]:
class MiniGPT(nn.Module):
    def __init__(self, vocab_size, embed_size, block_size, num_heads, num_layers, dropout):
        super().__init__()
        self.embedding = EmbeddingLayer(vocab_size, embed_size, block_size)
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_size, num_heads, block_size, dropout) for _ in range(num_layers)
        ])
        self.ln_f = nn.LayerNorm(embed_size)
        self.head = nn.Linear(embed_size, vocab_size, bias=False)

    def forward(self, x):
        x = self.embedding(x)          # 输入转为嵌入向量
        x = x.transpose(0, 1)          # 调整为 (seq_len, batch_size, embed_size)
        for block in self.transformer_blocks:
            x = block(x)               # 逐层处理
        x = self.ln_f(x)               # 最后归一化
        x = x.transpose(0, 1)          # 恢复为 (batch_size, seq_len, embed_size)
        logits = self.head(x)          # 输出词的概率分布
        return logits

### 第九步：初始化模型和优化器
- **设备**：优先使用 Mac M1 的 MPS 加速计算。
- **损失函数**：交叉熵损失，衡量预测词和真实词的差距。
- **优化器**：Adam 算法，根据损失调整模型参数。

**原理**：优化器是模型的“导航仪”，通过梯度下降逐步优化参数，让预测更准确。

In [12]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = MiniGPT(VOCAB_SIZE, EMBED_SIZE, BLOCK_SIZE, NUM_HEADS, NUM_LAYERS, DROPOUT).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(f"模型初始化完成，运行在 {device} 上")

模型初始化完成，运行在 mps 上


### 第十步：实现文本生成
生成函数测试模型的语言能力：
- 输入起始词，模型逐个预测后续词。
- 每次选择概率最高的词（贪婪解码），拼接成句子。

**原理**：生成过程展示了 Transformer 的自回归特性，体现了它对语言序列的理解。

In [13]:
def generate(model, vocab, start_text, max_length=10):
    """
    生成文本
    :param model: 训练好的模型
    :param vocab: 词汇表
    :param start_text: 起始文本
    :param max_length: 生成的最大长度
    :return: 生成的文本
    """
    model.eval()
    reverse_vocab = {idx: word for word, idx in vocab.items()}
    with torch.no_grad():
        start_tokens = list(jieba.cut(start_text, cut_all=False))
        indices = torch.tensor(tokens_to_indices(start_tokens, vocab), dtype=torch.long).unsqueeze(0).to(device)
        for _ in range(max_length):
            logits = model(indices)
            next_token = torch.argmax(logits[:, -1, :], dim=-1)
            indices = torch.cat([indices, next_token.unsqueeze(0)], dim=1)
        return "".join([reverse_vocab.get(idx.item(), '<UNK>') for idx in indices[0]])

### 第十一步：训练模型
训练是模型学习的过程：
- **前向传播**：输入序列，计算预测结果。
- **损失计算**：比较预测和真实目标，得出误差。
- **反向传播**：根据误差调整模型参数。
- **监控指标**：记录损失和困惑度（exp(loss)），评估模型性能。
- **生成测试**：每轮生成文本，观察学习效果。

**设计精髓**：Transformer 通过自注意力机制高效并行处理序列，反向传播让它逐步掌握语言规律。

In [14]:
def train(model, train_loader, epochs):
    """
    训练模型
    :param model: MiniGPT 模型
    :param train_loader: 训练数据加载器
    :param epochs: 训练轮数
    """
    train_losses = []
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_count = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()          # 清空梯度
            logits = model(inputs)         # 前向传播
            loss = criterion(logits.view(-1, VOCAB_SIZE), targets.view(-1))  # 计算损失
            loss.backward()                # 反向传播
            optimizer.step()               # 更新参数
            
            train_loss += loss.item()
            train_count += 1
        
        avg_train_loss = train_loss / train_count
        train_losses.append(avg_train_loss)
        train_perplexity = math.exp(avg_train_loss)  # 困惑度
        
        print(f"Epoch {epoch + 1}/{epochs}:")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train Perplexity: {train_perplexity:.2f}")
        print(f"  Generated: {generate(model, vocab, '人工智能是', max_length=10)}")
    
    # 可视化损失曲线
    plt.plot(range(1, epochs + 1), train_losses, label='Train Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.legend()
    plt.show()

### 第十二步：启动训练
- **加载数据**：从文件创建词汇表和数据集。
- **训练**：用 DataLoader 分批输入数据，开始学习。
- **保存和测试**：保存模型权重，生成最终文本。

**原理**：训练是 Transformer 从“零基础”到“语言专家”的过程，DataLoader 的随机打乱增强了泛化能力。

In [ ]:
file_path = "train.txt"  # 训练文件路径

print("构建词汇表...")
vocab = build_vocab_from_file(file_path)
dataset = TextDataset(file_path, BLOCK_SIZE, STRIDE, vocab)

train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f"训练数据量: {len(dataset)} 个序列")
print("开始训练...")

train(model, train_loader, EPOCHS)

torch.save(model.state_dict(), "mini_gpt.pth")
print("训练完成！模型保存为 'mini_gpt.pth'")

print("最终生成文本:", generate(model, vocab, "人工智能是", max_length=10))